## Working with databases in Python 

May 6, 2025

EDS 213 - Databases

In [1]:
# View working directory 
%pwd

'/Users/jcsibley/MEDS/eds-213-database/bren-eds213-data/workbench'

In [1]:
# Install duckdb 
# %pip install duckdb

import duckdb

In [8]:
# View database in directory 
%ls ~/MEDS/eds-213-database/bren-eds213-data/database/

10               schema-build-script.sql    tues-week5-slq-wrapup.sql
build-database*  species_test.csv           tues_week3.sql
database.db      thurs-week4.sql            tues_week4sql
database.sqlite  thurs_week3.sql
export_adsn/     tues-week5-exportdata.sql


In [9]:
# Connect to database 
conn = duckdb.connect("~/MEDS/eds-213-database/bren-eds213-data/database/database.db")

In [10]:
conn

In [11]:
cur = conn.cursor()

In [12]:
# Notice that this does get you the info you need 
cur.execute("SELECT * FROM Site Limit 3")

## Method 1: getting al at once using `fetchall`

In [13]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375)]

Cursors dont save any results; they're just a pass-through mechanism 

In [14]:
# This wont give you any info since you ran this above, now it is empty
cur.fetchall()

[]

Cursors always return list of tuples, even 1-tuples 

In [16]:
cur.execute("SELECT Code FROM Site LIMIT 3")
cur.fetchall()

[('barr',), ('burn',), ('bylo',)]

Pretty common to use a list comprehension to pull out the values more conceniently 

In [17]:
cur.execute("SELECT Code FROM Site LIMIT 3")

[t[0] for t in cur.fetchall()]

['barr', 'burn', 'bylo']

## Method 2: Getting one result at a time using `fetchone()`

In [18]:
cur.execute("SELECT Code FROM Site")
cur.fetchone()

('barr',)

In [19]:
cur.fetchone()

('burn',)

In [20]:
cur.fetchone()

('bylo',)

Still a tuple, but not a list of things 

## Method 3: Iterate over a cursor -- sadly not supported by duckdb

In [21]:
cur.execute("SELECT Code FROM Site")

for row in cur: 
    print(row)

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

This obviously didn't work, but it would work for other programs 

Can do things other then SELECTs

In [24]:
cur.execute("CREATE TEMP TABLE t AS SELECT * FROM Bird_nests WHERE Nest_ID LIKE '14%' ")

In [25]:
cur.fetchall()

[(99,)]

In [27]:
cur.execute("SELECT COUNT (*) FROM t")
# Name the columns beign requested 
cur.fetchone()[0]

99

In [28]:
cur.execute("SELECT Code, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('barr', 71.30000305175781, -156.60000610351562),
 ('burn', 55.20000076293945, -84.30000305175781),
 ('bylo', 73.19999694824219, -80.0)]

## Parameterized queries 

In [29]:
cur.execute("SELECT Code FROM Species LIMIT 3")
cur.fetchall()

[('agsq',), ('amcr',), ('amgp',)]

Hypothesize: we want to know the number of nests for each species (and were not going to use GROUP BY)

In [30]:
species = "agsq"
query = "SELECT COUNT (*) FROM Bird_nests WHERE Species = ?"

# Plug in 'species' where ever there is a question mark
cur.execute(query, [species])
cur.fetchall()

[(0,)]

Let's put two things together: query for all species, and loop over those species, getting the number of nest for each species 

In [33]:
cur.execute("SELECT Code FROM Species")
for row in cur.fetchall(): # In any other database, could iterate over cursor 
    species_code = row[0]
    cur2 = conn.cursor() # add another cursor 
    cur2.execute(query, [species_code])
    num_nests = cur2.fetchone()[0]
    print(f"Species {species_code} has {num_nests} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests
Species arfo has 0 nests
Species arte has 0 nests
Species basa has 0 nests
Species bbis has 0 nests
Species bbpl has 43 nests
Species bbsa has 0 nests
Species besw has 0 nests
Species bltu has 0 nests
Species brant has 0 nests
Species brbe has 0 nests
Species brle has 0 nests
Species btcu has 0 nests
Species btgo has 3 nests
Species cole has 0 nests
Species cora has 0 nests
Species cosn has 0 nests
Species crpl has 2 nests
Species cusa has 0 nests
Species dunl has 101 nests
Species eywa has 0 nests
Species glgu has 0 nests
Species goea has 0 nests
Species gwfg has 0 nests
Species gwgu has 0 nests
Species gwte has 0 nests
Species gyrf has 0 nests
Species herg has 3 nests
Species hore has 0 nests
Species hugo has 0 nests
Species kill has 0 nests
Species lalo has 33 nests
Species lbdo has 1 nests
Species lesa has 0 nests
Species leye has 0 nests
Species list has 0 nests
Species ltdu has 0 nests
Species ltja has 0 

There are lots of convenience functions and packages to use with duckdb

In [34]:
import pandas as pd

In [35]:
# Read in dataframe from database
# Give it query and connection to database
df = pd.read_sql("SELECT * FROM Site", conn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Gives a warning, but it does work! 

In [37]:
# View dataframe 
df

,Code,Site_name,Location,Latitude,Longitude,Area
0,barr,Barrow,"Alaska, USA",71.300003,-156.600006,220.399994
1,burn,Burntpoint Creek,"Ontario, Canada",55.200001,-84.300003,63.000000
2,bylo,Bylot Island,"Nunavut, Canada",73.199997,-80.000000,723.599976
3,cakr,Cape Krusenstern,"Alaska, USA",67.099998,-163.500000,54.099998
4,cari,Canning River Delta,"Alaska, USA",70.099998,-145.800003,722.000000
5,chau,Chaun River Delta,"Chukotka, Russia",68.800003,170.600006,248.199997
6,chur,Churchill,"Manitoba, Canada",58.700001,-93.800003,866.900024
7,coat,Coats Island,"Nunavut, Canada",62.900002,-82.500000,1239.099976
8,colv,Colville River Delta,"Alaska, USA",70.400002,-150.699997,324.799988
9,eaba,East Bay,"Nunavut, Canada",64.000000,-81.699997,1205.500000
